In [1]:
import os
import random
import shutil
from glob import glob
from ultralytics import YOLO

# === Paths ===
ROOT = os.getcwd() 
DATA_DIR = os.path.abspath(os.path.join(ROOT, "../data"))
TILE_DIR = os.path.join(DATA_DIR, "tiles")
LABEL_DIR = os.path.join(DATA_DIR, "labels")

DATASET_DIR = os.path.abspath(os.path.join(ROOT, "../dataset"))
os.makedirs(DATASET_DIR, exist_ok=True)

# === Parameters ===
TRAIN_SPLIT = 0.8
IMG_SIZE = 512
EPOCHS = 50
MODEL_SIZE = "yolov8n.pt"


In [2]:
# === Make folders ===
for split in ["train", "val"]:
    os.makedirs(os.path.join(DATASET_DIR, f"images/{split}"), exist_ok=True)
    os.makedirs(os.path.join(DATASET_DIR, f"labels/{split}"), exist_ok=True)

# === Load and split ===
images = sorted(glob(os.path.join(TILE_DIR, "*.png")))
random.shuffle(images)
split_idx = int(len(images) * TRAIN_SPLIT)

train_imgs = images[:split_idx]
val_imgs = images[split_idx:]

def copy_split(img_list, split):
    for img_path in img_list:
        filename = os.path.basename(img_path)
        name = os.path.splitext(filename)[0]
        label_path = os.path.join(LABEL_DIR, f"{name}.txt")

        if os.path.exists(label_path):
            shutil.copy(img_path, os.path.join(DATASET_DIR, f"images/{split}/{filename}"))
            shutil.copy(label_path, os.path.join(DATASET_DIR, f"labels/{split}/{name}.txt"))

copy_split(train_imgs, "train")
copy_split(val_imgs, "val")


In [3]:
# === Create data.yaml ===
yaml_content = f"""
path: {DATASET_DIR}
train: images/train
val: images/val

names:
  0: ship
"""

with open(os.path.join(DATASET_DIR, "data.yaml"), "w") as f:
    f.write(yaml_content)

In [2]:
# === Train YOLOv8 ===
print("\n✅ Training YOLOv8...\n")
model = YOLO(MODEL_SIZE)
model.train(data=os.path.join(DATASET_DIR, "data.yaml"), epochs=EPOCHS, imgsz=IMG_SIZE, batch=1, device='cpu')


✅ Training YOLOv8...

Ultralytics 8.3.111  Python-3.9.21 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-1135G7 2.40GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=d:\Github repos\ship-detection\dataset\data.yaml, epochs=50, time=None, patience=100, batch=1, imgsz=512, save=True, save_period=-1, cache=False, device=cpu, workers=8, project=None, name=train14, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, s

train: Scanning D:\Github repos\ship-detection\dataset\labels\train.cache... 33 images, 0 backgrounds, 0 corrupt: 100%|██████████| 33/33 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.10.0 ms, read: 387.1100.6 MB/s, size: 212.5 KB)



val: Scanning D:\Github repos\ship-detection\dataset\labels\val.cache... 22 images, 0 backgrounds, 0 corrupt: 100%|██████████| 22/22 [00:00<?, ?it/s]


Plotting labels to runs\detect\train14\labels.jpg... 


: 

In [ ]:
# === Evaluate model ===
model.val()